In [68]:
import mlflow
mlflow.autolog()

2023/12/29 15:48:49 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2023/12/29 15:48:49 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.
2023/12/29 15:48:49 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.


Importation des données

In [69]:
import pandas as pd
ct = 'QueryResults (1).csv'
df = pd.read_csv(ct)
df


,Id,Title,Body,Tags,Score,ViewCount,AnswerCount
0,20485075,How do I define multiple conditions on a join ...,"<p>Ok, lets say I have order and items tables....",<java><jpa><playframework><playframework-2.0><...,1,1984,2
1,20485079,What is the equivalent of a C# gridview in jav...,"<p>I am trying to learn java, I am working on ...",<c#><java><asp.net><swing><gridview>,2,12333,1
2,20485143,"Listview cursor adapter, always get first item...",<p>I have this piece of code for listview usin...,<android><listview><android-listview><simplecu...,1,1075,2
3,20485144,OpenSSL RSA-2048 unencrypted block is longer t...,<p>I am using the OpenSSL library in order to ...,<c><encryption><cryptography><openssl><rsa>,0,527,1
4,20485176,Returning char into main method,<p>The goal of my program is kind of like whee...,<java><eclipse><string><methods><char>,0,1278,3
...,...,...,...,...,...,...,...
49995,22814994,StructureMap use specific instance of type per...,"<p>Is there a way, using StructureMap (the Dep...",<c#><asp.net><dependency-injection><inversion-...,0,369,2
49996,22815009,Add a reference column migration in Rails 4,<p>A user has many uploads. I want to add a co...,<ruby-on-rails><ruby-on-rails-4><foreign-keys>...,368,397557,8
49997,22815010,Hibernate mapping with group by clause?,<p>I have <code>@OneToMany</code> mapping in a...,<java><sql><hibernate><group-by><hibernate-map...,0,1755,1
49998,22815022,java.lang.NoClassDefFoundError: android.suppor...,<p>I have a class that extends <code>android.s...,<java><android><eclipse><android-fragments><an...,0,1441,1


Filtre des tags

In [70]:
df['Tags'] = df['Tags'].str.replace('><', ',').str.replace('<', '').str.replace('>', '')

from collections import Counter
# Fréquence des mots dans 'Tags'
all_tags = df['Tags'].str.split(',').explode().tolist()
tag_counts = Counter(all_tags)

# Trouver les 50 mots les plus fréquents
top_50_tags = [tag for tag, _ in tag_counts.most_common(50)]

# Filtrer les listes de mots pour ne contenir que les 50 mots les plus fréquents
def filter_tags(tags):
    return [tag for tag in tags if tag in top_50_tags]

df['Tags'] = df['Tags'].str.split(',').apply(filter_tags)

# Filtrer les lignes où 'Tags' est vide après le filtrage
df = df[df['Tags'].apply(len) > 0]

df['nb_tags'] = df['Tags'].apply(lambda x: len(x))
df = df[df['nb_tags']==3].reset_index(drop=True)

Nettoyage du texte

In [71]:
import pickle
import nltk

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

custom_punkt_path = '/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/corpora/punkt'
punkt_path = nltk.data.find(f'{custom_punkt_path}/english.pickle')

with open(punkt_path, 'rb') as file:
    punkt_model = pickle.load(file)
nltk.data.path.append(custom_punkt_path)

with open('/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/corpora/stopwords/english', 'r') as file:
    custom_stopwords = file.read().splitlines()
    
# Nettoyage
def strip_html_bs(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

def tokenizer_fct(sentence):
    """Division de mots en texte + suppression de certains caractères"""
    sentence_clean = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ')
    word_tokens = sentence_clean.split()
    return word_tokens

stop_w = custom_stopwords + ['[', ']', ',', '.', ':', '?', '(', ')','<','>','~',':']

def stop_word_filter_fct(list_words):
    """Suppression de mots sans information+ ponctuations"""
    filtered_w = [w for w in list_words if not w in stop_w]
    filtered_w2 = [w for w in filtered_w if len(w) > 2]
    return filtered_w2

def lower_start_fct(list_words):
    """Conversion en lettres minuscules et suppression de préfixes indésirables"""
    lw = [w.lower() for w in list_words if (not w.startswith("@")) and (not w.startswith("#")) and (not w.startswith("http"))]
    return lw

def lemma_fct(list_words):
    """lemmatisation"""
    lemmatizer = WordNetLemmatizer()
    lem_w = [lemmatizer.lemmatize(w) for w in list_words]
    return lem_w

def transform_bow_fct(desc_text):
    """fonction de transformation"""
    text_stripped = strip_html_bs(desc_text)
    word_tokens = tokenizer_fct(text_stripped)
    lw = lower_start_fct(word_tokens)
    sw = stop_word_filter_fct(lw)
    transf_desc_text = ' '.join(sw)
    return transf_desc_text

# Prétraitement
df['Cleaned_Body'] = df['Body'].apply(transform_bow_fct)


In [76]:
df['Body'][1000]

'<p>VS2008 Code Analysis will flag a spelling mistake in an identifier using the <code>IdentifiersShouldBeSpelledCorrectly</code> warning type.</p>\n\n<p>This process is using an American dictionary by default because words are being flagged that are correctly spelt using the British spelling. For example, words like "Organisation" and "Customisation", etc...</p>\n\n<p>I am aware that you can create your own custom Xml dictionary files that contain any words you don\'t want to be flagged, however, can anyone tell me if you can configure Code Analysis to use a different default (or additional) dictionary from those available in Windows?</p>\n'

In [77]:
df['Cleaned_Body'][1000]

'vs2008 code analysis flag spelling mistake identifier using identifiersshouldbespelledcorrectly warning type. process using american dictionary default words flagged correctly spelt using british spelling. example, words like "organisation" "customisation", etc... aware create custom xml dictionary files contain words want flagged, however, anyone tell configure code analysis use different default (or additional) dictionary available windows?'

Encoding TF - IDF

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_array = tfidf_vectorizer.fit_transform(df['Cleaned_Body']).toarray()

# Liste des termes correspondant aux colonnes de la matrice
feature_names = tfidf_vectorizer.get_feature_names_out()
# Scores de chaque mot
tfidf_df = pd.DataFrame(tfidf_array, columns=feature_names)

# Concaténer les DataFrames sans utiliser de sample
df.reset_index(drop=True, inplace=True)
df2 = pd.concat([df, tfidf_df], axis=1)

Modèle supervisé

In [13]:
import random as rd
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


rd.seed(0)

def modelisation(enc,name):

    # Diviser les données pour l'entraînement et le test
    X_train, X_test, y_train, y_test = train_test_split(enc, df['Tags'], test_size=0.2, random_state=42)
    y_train = [y[rd.randint(0, 2)] for y in y_train]
    y_test = [y[rd.randint(0, 2)] for y in y_test]

    # Initialisation et entrainement du modèle
    with mlflow.start_run(run_name= 'supervised_'+name+'_final') as run:
        naive_bayes = MultinomialNB(fit_prior=True)
        naive_bayes.fit(X_train, y_train)
        mlflow.sklearn.log_model(
            sk_model=naive_bayes,
            artifact_path="model_final",
            registered_model_name="supervised_tfidf_vec_final",
        )
        mlflow.sklearn.log_model(
            sk_model=tfidf_vectorizer, 
            artifact_path="tfidf_vectorizer__final", 
            registered_model_name="supervised__final")


    # Prédictions
    predicted_tags = naive_bayes.predict(X_test)
    accuracy = accuracy_score(y_test, predicted_tags)

    return("La précision du modèle supervisé avec ",name," est de ",round(accuracy,2)*100," % ")

modelisation(tfidf_df,'tf_idf')

2023/12/29 14:14:46 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2922: UserWarning:

The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.

Successfully registered model 'supervised_tfidf_vec_final'.
Created version '1' of model 'supervised_tfidf_vec_final'.
2023/12/29 14:15:00 WARNING mlflow.sklearn: Model was missing function:

('La précision du modèle supervisé avec ', 'tf_idf', ' est de ', 21.0, ' % ')

Evaluation du drift avec evidently

In [14]:
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

X_train, X_test, y_train, y_test = train_test_split(tfidf_df, df['Tags'], test_size=0.2, random_state=42)
y_train = [y[rd.randint(0, 2)] for y in y_train]
y_test = [y[rd.randint(0, 2)] for y in y_test]

df_y_train = pd.DataFrame(y_train, columns=['Target'])
df_y_test = pd.DataFrame(y_test, columns=['Target'])
"""
data_stability= TestSuite(tests=[DataStabilityTestPreset(),])
data_stability.run(current_data=df_y_train, reference_data=df_y_test, column_mapping=None)
data_stability 
"""
data_drift_report = Report(metrics=[DataDriftPreset(),])

data_drift_report.run(current_data=df_y_train, reference_data=df_y_test, column_mapping=None)
data_drift_report

Extraction de caractéristiques avec Word2Vec

In [15]:
from gensim.models import Word2Vec
import numpy as np

sentences = [text.split() for text in df['Cleaned_Body']]

# Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Dictionnaire de mots vers vecteurs
word_vectors = word2vec_model.wv

# Obtention du vecteur d'un document en moyennant les vecteurs de ses mots
def document_vector(word_vectors, doc):
    # Supprimer les mots absents du vocabulaire du modèle
    doc = [word for word in doc if word in word_vectors.key_to_index]
    if len(doc) > 0:
        return np.mean(word_vectors[doc], axis=0)
    else:
        return np.zeros(word_vectors.vector_size)

word2vec_features = np.array([document_vector(word_vectors, doc) for doc in sentences])

word2vec_df = pd.DataFrame(word2vec_features, columns=[f'w2v_{i}' for i in range(word2vec_features.shape[1])])

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
word2vec_df = scaler.fit_transform(word2vec_df)
modelisation(word2vec_df,'Word2Vec')

2023/12/29 14:15:15 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'list' object has no attribute 'flatten'
/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2922: UserWarning:

The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.

Registered model 'supervised_tfidf_vec_final' already exists. Creating a new version of this model...
Created version '2' of model 'supervised_tfidf_vec_final'.
2023/12/29 14:15:28 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
Registered model 'supervised__final' already exists. Cre

('La précision du modèle supervisé avec ', 'Word2Vec', ' est de ', 11.0, ' % ')

Extraction de caractéristiques avec USE

In [22]:
import tensorflow_hub as hub
import pandas as pd
import numpy as np

# Modèle Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Obtention du vecteur d'un document
def get_use_embedding(text):
    # Convertir le texte en embeddings
    embeddings = embed([text])
    return np.array(embeddings).flatten()


use_features = np.array([get_use_embedding(text) for text in df['Cleaned_Body']])

use_df = pd.DataFrame(use_features, columns=[f'use_{i}' for i in range(use_features.shape[1])])
use_df = scaler.fit_transform(use_df)

modelisation(word2vec_df,'USE')

2023/12/29 14:23:41 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'list' object has no attribute 'flatten'
/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2922: UserWarning:

The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.

Registered model 'supervised_tfidf_vec_final' already exists. Creating a new version of this model...
Created version '3' of model 'supervised_tfidf_vec_final'.
2023/12/29 14:23:56 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
Registered model 'supervised__final' already exists. Cre

('La précision du modèle supervisé avec ', 'USE', ' est de ', 11.0, ' % ')

Extraction de caractéristiques avec BERT

In [100]:
from transformers import BertTokenizer, TFBertModel
import pandas as pd
import numpy as np


# Modèle BERT et le tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')


# Obtention du vecteur d'un document
def get_bert_embedding(text):
    # Tokenizer le texte et obtenir les embeddings
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True)
    outputs = model(**inputs)
    # Prendre la sortie correspondant au [CLS] token
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    return np.array(embeddings).flatten()

texts = df['Cleaned_Body']

bert_features = np.array([get_bert_embedding(text) for text in texts])

bert_df = pd.DataFrame(bert_features, columns=[f'bert_{i}' for i in range(bert_features.shape[1])])
bert_df = scaler.fit_transform(bert_df)

modelisation(bert_df,'BERT')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

('La précision du modèle supervisé avec ', 'BERT', ' est de ', 12.0, ' % ')